# This notebook can be executed after getRegionalBox.ipynb

In [1]:
using Pkg
using Metal

cd(@__DIR__)
Pkg.activate("../")
ParamFile = "../config/testparam.csv"  # maybe GeoPoints and planet1D should be fusioned

# batchGPU should be at this level (I have not made it as a module yet, since the choice of Metal/CUDA should be done in a manual way)
include("../src/batchFiles/batchGPU.jl")


include("../src/commonBatchs.jl")
include("../src/flexOPT.jl")
include("../src/planet1D.jl")
include("../src/GeoPoints.jl")

using .commonBatchs, .flexOPT, .planet1D, .GeoPoints

  Activating project at `~/Documents/Github/flexOPT`


devs = Metal.devices() = Metal.MTL.MTLDeviceInstance[Metal.MTL.MTLDeviceInstance (object of type AGXG13XDevice)]
→ Using Metal backend (1 device(s))
Selected backend type: MetalBackend
  1.790276 seconds (3.83 M allocations: 173.479 MiB, 2.64% gc time, 99.98% compilation time)


# give me a model of material variables (like seismic models)

In [2]:
# what you need as a model:

boxGrids = lazyProduceOrLoad("MarmousiCoordInfo"); # for a big model, I do not recommend to store these data

seismicModelMarmousi=lazyProduceOrLoad("seismicModelMarmousi");

This is the laziest ProduceOrLoad
Loading from ./tmp/MarmousiCoordInfo.jld2
This is the laziest ProduceOrLoad
Loading from ./tmp/seismicModelMarmousi.jld2


# choose your physics and get the semi symbolic OPT expression(s) 

In [3]:
famousEquationType="2DacousticTime"
@show myEquationInside=famousEquations(famousEquationType)


myEquationInside = famousEquations(famousEquationType) = (exprs = Differential(t)(Differential(t)(u(x, y, t))) - (Differential(x)(Differential(x)(u(x, y, t))) + Differential(y)(Differential(y)(u(x, y, t))))*(v(x, y)^2), fields = u(x, y, t), vars = v(x, y), extexprs = f(x, y, t), extfields = f(x, y, t), extvars = 1, coordinates = (x, y, t), ∂ = Any[Differential(x), Differential(y), Differential(t)], ∂² = Any[Differential(x) ∘ Differential(x), Differential(y) ∘ Differential(y), Differential(t) ∘ Differential(t)])


(exprs = Differential(t)(Differential(t)(u(x, y, t))) - (Differential(x)(Differential(x)(u(x, y, t))) + Differential(y)(Differential(y)(u(x, y, t))))*(v(x, y)^2), fields = u(x, y, t), vars = v(x, y), extexprs = f(x, y, t), extfields = f(x, y, t), extvars = 1, coordinates = (x, y, t), ∂ = Any[Differential(x), Differential(y), Differential(t)], ∂² = Any[Differential(x) ∘ Differential(x), Differential(y) ∘ Differential(y), Differential(t) ∘ Differential(t)])

In [4]:
Δnum = (1.0,1.0,1.0) # this should be in the same order as myEquationInside.coordinates 

# or
 
Δnum = (boxGrids.Δx,boxGrids.Δz,1.0) # hahaha this Δnum is not very good since there is no reason for the grids to be rectangular with this isotropic medium but let's try


(11.485642946317103, 4.444444444444445, 1.0)

# hereafter are the parameters which are important for modellers but maybe not interesting (so default numbers are given)

In [5]:
# concerning the order of test B-spline functions (-1 denotes a pulse function)
orderBtime=1
orderBspace=1
# the number of neighbouring points used to construct operators (except the point itself)
pointsInSpace=2
pointsInTime=2
# in order to control higher order partials, we need the supplemantary order > 0, for an ill-posed inversion of Taylor coefficients
supplementaryOrder=2
# B-spline order for the interpolated Taylor expansion
WorderBspace=1
WorderBtime=1

TaylorOptions=(WorderBtime=WorderBtime,WorderBspace=WorderBspace,supplementaryOrder=supplementaryOrder)
trialFunctionsCharacteristics=(orderBtime=orderBtime,orderBspace=orderBspace,pointsInSpace=pointsInSpace,pointsInTime=pointsInTime)



(orderBtime = 1, orderBspace = 1, pointsInSpace = 2, pointsInTime = 2)

In [6]:
OPTcoefConfig = @strdict myEquationInside=myEquationInside Δnum=Δnum TaylorOptions=TaylorOptions trialFunctionsCharacteristics=trialFunctionsCharacteristics 
coefs = myProduceOrLoad(OPTobj,OPTcoefConfig,"OPTcoefs") #attention! without GPU, OPTobj will take a lot of time (for >2D)

Dict{String, Any} with 2 entries:
  "output"    => (Ajiννᶜs = Any[Float32[0.0329029 -0.036751 … -5.71263f-5 5.981…
  "gitcommit" => "544530c1b2ef3a2662914b222240b76b4015bdbc-dirty"

In [7]:
@show size(coefs["output"].AjiννᶜUs)
#@show coefs["output"].ΓjkννᶜFs

size((coefs["output"]).AjiννᶜUs) = (1,)


(1,)

# now we construct numerical operators based on the semi-symbolic rules that we constructed above

In [8]:
models=[] # you might need to make this empty tuple first, otherwise one-member tuple can be misinterpreted
models=push!(models,seismicModelMarmousi.Vpv.*1.e3) # 1.e3 from km/s -> m/s

# in this example, the material variable is just v and we use Vpv 

# put fake Nt here for quasi-numerical operator construction


1-element Vector{Any}:
 [4101.554566049527 3776.772199095795 … 1500.0 1500.0; 4117.178345441763 3827.7831476265005 … 1500.0 1500.0; … ; 4590.5189384990945 4503.897944720057 … 1500.0 1500.0; 4590.5189384990945 4445.277712678698 … 1500.0 1500.0]

In [9]:
models=push!(models,seismicModelMarmousi.Vpv.*1.e3) 
modelPoints = getModelPoints(models[1],pointsInTime,coefs["output"].utilities.timeMarching)#reference model dimension (this is important to explicitly define since the dimension/size of material variable array(s) can differ from the model domain)

(802, 676, 3)

In [10]:
IneedExternalSources = true
maskedRegionForSourcesInSpace = nothing
# if IneedExternalSources and if the source region is localised in space then
maskedRegionForSourcesInSpace  = Array{CartesianIndex,1}(undef,0) # it is important to decalre the type of this
maskedRegionForSourcesInSpace = push!(maskedRegionForSourcesInSpace, CartesianIndex(modelPoints[1:end-1].÷2))# in Ndimension (or Ndimension  - 1 if timeMarching)
# in this example, I put a point source at the centre of the model space

forceModels =((1.0)) # if your model does not have anything special material parameters then it's how it's written

1.0

In [15]:
include("../src/flexOPT.jl")

Main.flexOPT

In [ ]:
lhsConfigurations = @strdict semiSymbolicCoefs=coefs models modelName="MarmousiVp" modelPoints myEquationInside maskedRegion=nothing
numOperators = myProduceOrLoad(constructingNumericalDiscretisedEquations,lhsConfigurations,"numOperators","lhs")